# Classification Tree Optimization Project

## Course Information

This project is part of the Master's Degree in Computer Engineering course, specifically for the "Optimization" class taught by Professor Francesca Maggioni.

## Assignment

The task for this course is to **implement one of the classification trees** presented in the paper by Bertsimas and Dunn and apply it to a real-life dataset discussed in the article.

### Paper Reference

- **Title**: Optimal Classification Trees
- **Authors**: Dimitris Bertsimas and Julia Dunn
- **Journal**: Machine Learning
- **Volume**: 106
- **Pages**: 1039–1082
- **Year**: 2017
- **DOI**: [10.1007/s10994-017-5633-9](https://doi.org/10.1007/s10994-017-5633-9)

### Objective

The objective of this project is to:

1. Implement a classification tree based on the methodology described in the paper.
2. Apply the implemented model to a real-world dataset to evaluate its performance and effectiveness.

### Overview of the Implementation

In this notebook, we will:

1. **Load and Prepare Data**: Import the dataset and preprocess it for use in the classification tree model.
2. **Standardize the Data**: Ensure that the data is scaled appropriately for the model.
3. **Split the Dataset**: Divide the data into training and test sets for model evaluation.
4. **Configure the Model**: Set up the parameters and constants for the classification tree model.
5. **Train and Tune the Model**: Train the model using the training data and fine-tune it for optimal performance.
6. **Compare Different Solvers**: Evaluate the performance of various solvers in solving the classification tree model.
7. **Evaluate and Report Results**: Assess the model’s performance on both training and test data and present the results.

By following these steps, we aim to understand the application of optimization techniques in classification problems and gain insights into the performance of classification trees on real-world data.



## Setup Instructions

To ensure a smooth development process, it is recommended to use a virtual environment. This helps manage dependencies and avoid conflicts. You can choose between `venv` or `conda` for creating the virtual environment.

### Using `venv`

1. **Create a virtual environment**:
    ```bash
    python -m venv myenv
    ```

2. **Activate the virtual environment**:
    - On Windows:
      ```bash
      myenv\Scripts\activate
      ```
    - On macOS and Linux:
      ```bash
      source myenv/bin/activate
      ```

3. **Install dependencies**:
    ```bash
    pip install -r requirements.txt
    ```

### Using `conda`

1. **Create a new conda environment** with Python 3.12:
    ```bash
    conda create -n myenv python=3.12
    ```

2. **Activate the conda environment**:
    ```bash
    conda activate myenv
    ```

3. **Install dependencies**:
    ```bash
    pip install -r requirements.txt
    ```

### Python Version

Ensure you are using **Python 3.12** for compatibility with the dependencies and the project code.

### Setting constants and model parameters

In this section, we define key constants and parameters that will be used throughout the model training and evaluation process.
These include the random seed for reproducibility, dataset split ratios, and model-specific parameters.

In [1]:
# For reproducibility
SEED = 26

# Split parameters
TRAIN_SIZE = 0.1
TEST_SIZE = 1 - TRAIN_SIZE

# Model parameters
ALPHA = 5 # complexity
MIN_SAMPLES_PER_LEAF = 2 # minimum number of samples per leaf
MAX_DEPTH = 3 # max depth of the tree


### Installing required packages and importing libraries

In this section, we install the necessary packages and import the required libraries for our project, this includes specific versions for reproducibility and compatibility.

In [2]:
# When pyomo will support numpy 2.0, we will update the version
%pip install numpy==1.26.4 \
    scipy \
    matplotlib \
    scikit-learn \
    ucimlrepo \
    pyomo==6.7.3 --quiet

import numpy as np
import time
import pyomo.environ as pyo
import importlib
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from src import MIOTree
from sklearn import tree

Note: you may need to restart the kernel to use updated packages.


### Utility functions

This section defines two utility functions: one for standardizing data and another for printing the confusion matrix.

In [3]:
def standardize(x):
    """Standardize the original data points (mean 0 and std dev 1)."""
    x = x - np.mean(x)
    x = x / np.std(x)
    return x

def print_confusion_matrix(classes, confusion_matrix):
    """Print the confusion matrix."""
    print('Confusion Matrix:')
    print ('Act/Pred\t(0)\t(1)')
    for index in range(len(confusion_matrix)):
        print(f'({classes[index]})\t\t', end='')
        for val in confusion_matrix[index]:
            print(val, end='\t')
        print()

def extract_leaf_predictions(model):
    classes = np.unique(model.y_train)

    num_leaf_nodes = len(model.pyomo_model.leaf_nodes)
    leaf_predictions = [None] * num_leaf_nodes

    class_index_to_label = {i: classes[i] for i in model.pyomo_model.classes_indices}


    for i in model.pyomo_model.classes_indices:
        for j in model.pyomo_model.leaf_nodes:
            if model.pyomo_model.c[i, j].value == 1:
                leaf_index = j - num_leaf_nodes
                if 0 <= leaf_index < num_leaf_nodes:
                    leaf_predictions[leaf_index] = class_index_to_label[i]

    return leaf_predictions


### Fetching and processing the dataset

In this section, we fetch a dataset from the UCI Machine Learning Repository, process it into NumPy arrays, and print out metadata and variable information.

In [4]:
# fetch dataset 
spambase = fetch_ucirepo(id=94) 
  
# data (as pandas dataframes) 
X = spambase.data.features
y = spambase.data.targets

# convert to numpy
X = X.to_numpy()
y = y.to_numpy()
  
# metadata 
print(spambase.metadata) 
  
# variable information 
print(spambase.variables) 

{'uci_id': 94, 'name': 'Spambase', 'repository_url': 'https://archive.ics.uci.edu/dataset/94/spambase', 'data_url': 'https://archive.ics.uci.edu/static/public/94/data.csv', 'abstract': 'Classifying Email as Spam or Non-Spam', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 4601, 'num_features': 57, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1999, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C53G6X', 'creators': ['Mark Hopkins', 'Erik Reeber', 'George Forman', 'Jaap Suermondt'], 'intro_paper': None, 'additional_info': {'summary': 'The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...\n\nThe classification task for this dataset is to determine whether a given email is spam or not.\n\t\nOur collecti

### Data standardization and train-test split

In this section, we standardize the dataset and split it into training and test sets.

In [5]:
X_std = standardize(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_std, 
    y, 
    train_size=TRAIN_SIZE,
    test_size=TEST_SIZE,
    random_state=SEED)

### Creating and tuning the model

In this section, we create an instance of the `MIOTree` model using the defined parameters and then tune the model using the test data.


In [6]:
model = MIOTree.MIOTree(
    alpha=ALPHA, 
    min_samples_per_leaf=MIN_SAMPLES_PER_LEAF, 
    max_depth=MAX_DEPTH,
    X_train=X_train, 
    y_train=y_train)

tuned_model = model.tune(X_test, y_test)

accuracy = tuned_model.calculate_accuracy()
print(f'Train Accuracy: {accuracy * 100}%')

confusion_matrix = tuned_model.calculate_confusion_matrix()
print_confusion_matrix(np.unique(model.y_train), confusion_matrix)

accuracy = tuned_model.calculate_accuracy(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100}%')

CART	Depth: 2	Alpha: 2	Accuracy: 0.8782608695652174	Duration: 0.001528024673461914
MIO	Depth: 2	Alpha: 2	Accuracy: 0.6260869565217392	Duration: 2.005417823791504
CART	Depth: 2	Alpha: 3	Accuracy: 0.8782608695652174	Duration: 0.0015611648559570312
MIO	Depth: 2	Alpha: 3	Accuracy: 0.6260869565217392	Duration: 1.7708189487457275
CART	Depth: 2	Alpha: 4	Accuracy: 0.8782608695652174	Duration: 0.0016911029815673828
MIO	Depth: 2	Alpha: 4	Accuracy: 0.6260869565217392	Duration: 2.0718252658843994
CART	Depth: 2	Alpha: 5	Accuracy: 0.8782608695652174	Duration: 0.0017359256744384766
MIO	Depth: 2	Alpha: 5	Accuracy: 0.6260869565217392	Duration: 2.1114439964294434
CART	Depth: 2	Alpha: 6	Accuracy: 0.8782608695652174	Duration: 0.0016360282897949219
MIO	Depth: 2	Alpha: 6	Accuracy: 0.6260869565217392	Duration: 2.1748416423797607
CART	Depth: 2	Alpha: 7	Accuracy: 0.8782608695652174	Duration: 0.0015730857849121094
MIO	Depth: 2	Alpha: 7	Accuracy: 0.6260869565217392	Duration: 2.2070960998535156
CART	Depth: 3	Alph

### Comparing different solvers

In this section, we compare the performance of different solvers on the `MIOTree` model by solving the model with each solver and evaluating its accuracy on both training and test datasets, the solvers compared are `gurobi`, `glpk`, and `ipopt`.


In [7]:
# Comparisons
results = []
max_easy_depth = 4
for solver in ['gurobi', 'ipopt']:
    model = MIOTree.MIOTree(
        alpha=ALPHA, 
        min_samples_per_leaf=MIN_SAMPLES_PER_LEAF, 
        max_depth=max_easy_depth,
        X_train=X_train, 
        y_train=y_train)

    print(f"Solver: {solver}")
    result = {
        'model': model,
    }
    result['solver'] = model.solve(solver)
    results.append(result)
    print(result['solver'])
    
    """ accuracy = model.calculate_accuracy()
    print(f'Train Accuracy: {accuracy * 100}%')
    
    confusion_matrix = model.calculate_confusion_matrix()
    print_confusion_matrix(np.unique(model.y_train), confusion_matrix)

    accuracy = model.calculate_accuracy(X_test, y_test)
    print(f'Test Accuracy: {accuracy * 100}%') """

    leaf_predictions = extract_leaf_predictions(model)
    model.tree.print_tree(leaf_predictions)



Solver: gurobi

Problem: 
- Name: x1
  Lower bound: 0.5972222222222222
  Upper bound: 0.5972222222222222
  Number of objectives: 1
  Number of constraints: 37464
  Number of variables: 8357
  Number of binary variables: 8278
  Number of integer variables: 8342
  Number of continuous variables: 15
  Number of nonzeros: 1782432
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 4.950109004974365
  Error rc: 0
  Time: 5.574570894241333
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Root: (1) None
    L--- (2) None
        L--- (4) None
            L--- (8) None
                L--- (16) 0
                R--- (17) None
            R--- (9) None
                L--- (18) None
                R---